# 0.3 — Baseline Models
Train simple baseline classifiers to predict trade outcome (win/loss).

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import (accuracy_score, classification_report,
                              confusion_matrix, roc_auc_score)
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

BASE = '.'
TRADERS = ['calm_trader', 'loss_averse_trader', 'overtrader', 'revenge_trader']
LABELS  = {'calm_trader':'Calm','loss_averse_trader':'Loss Averse',
           'overtrader':'Overtrader','revenge_trader':'Revenge'}


## Load Train/Val/Test Splits

In [ ]:
splits = {}
for t in TRADERS:
    proc = f'{BASE}/{t}/data/processed'
    splits[t] = {
        'train': pd.read_csv(f'{proc}/{t}_train.csv'),
        'val':   pd.read_csv(f'{proc}/{t}_val.csv'),
        'test':  pd.read_csv(f'{proc}/{t}_test.csv'),
    }
    for s, df in splits[t].items():
        print(f"{t} {s}: {len(df)} rows, win_rate={df['win'].mean():.3f}")


## Feature Selection

In [ ]:
FEATURES = ['quantity', 'entry_price', 'exit_price',
           'asset_encoded', 'side_encoded',
           'year', 'month', 'day', 'hour', 'minute',
           'price_change', 'price_change_pct']
TARGET = 'win'

def get_XY(df):
    df = df.dropna(subset=FEATURES + [TARGET])
    return df[FEATURES], df[TARGET]


## Define Models

In [ ]:
MODELS = {
    'Dummy (Majority)':   DummyClassifier(strategy='most_frequent'),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree':       DecisionTreeClassifier(max_depth=5, random_state=42),
    'Random Forest':       RandomForestClassifier(n_estimators=100, max_depth=6, random_state=42, n_jobs=-1),
}


## Train & Evaluate on Validation Set

In [ ]:
results = {}

for t in TRADERS:
    X_train, y_train = get_XY(splits[t]['train'])
    X_val,   y_val   = get_XY(splits[t]['val'])

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_val_s   = scaler.transform(X_val)

    results[t] = {}
    for name, model in MODELS.items():
        model.fit(X_train_s, y_train)
        preds = model.predict(X_val_s)
        proba = model.predict_proba(X_val_s)[:, 1] if hasattr(model, 'predict_proba') else preds
        acc   = accuracy_score(y_val, preds)
        auc   = roc_auc_score(y_val, proba) if len(set(y_val)) > 1 else 0.5
        results[t][name] = {'acc': acc, 'auc': auc, 'model': model, 'scaler': scaler}
        print(f"{LABELS[t]} | {name:25s} → Acc: {acc:.4f}  AUC: {auc:.4f}")
    print()


## Results Heatmap

In [ ]:
model_names = list(MODELS.keys())
trader_names = [LABELS[t] for t in TRADERS]

acc_matrix = np.array([[results[t][m]['acc'] for m in model_names] for t in TRADERS])
auc_matrix = np.array([[results[t][m]['auc'] for m in model_names] for t in TRADERS])

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
for ax, mat, title in zip(axes, [acc_matrix, auc_matrix], ['Accuracy', 'ROC-AUC']):
    im = ax.imshow(mat, cmap='RdYlGn', vmin=0.4, vmax=0.7)
    ax.set_xticks(range(len(model_names))); ax.set_xticklabels(model_names, rotation=20, ha='right', fontsize=9)
    ax.set_yticks(range(len(trader_names))); ax.set_yticklabels(trader_names)
    ax.set_title(f'Validation {title}')
    for i in range(len(trader_names)):
        for j in range(len(model_names)):
            ax.text(j, i, f'{mat[i,j]:.3f}', ha='center', va='center', fontsize=9, fontweight='bold')
    plt.colorbar(im, ax=ax)
plt.tight_layout(); plt.show()


## Best Model — Test Set Evaluation

In [ ]:
print("=" * 60)
for t in TRADERS:
    best_name = max(results[t], key=lambda m: results[t][m]['auc'])
    best = results[t][best_name]

    X_test, y_test = get_XY(splits[t]['test'])
    X_test_s = best['scaler'].transform(X_test)
    preds = best['model'].predict(X_test_s)

    print(f"\n{LABELS[t]} — Best Model: {best_name}")
    print(classification_report(y_test, preds, target_names=['Loss','Win']))
    print("=" * 60)


## Confusion Matrices

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
for ax, t in zip(axes.flat, TRADERS):
    best_name = max(results[t], key=lambda m: results[t][m]['auc'])
    best = results[t][best_name]
    X_test, y_test = get_XY(splits[t]['test'])
    X_test_s = best['scaler'].transform(X_test)
    preds = best['model'].predict(X_test_s)
    cm = confusion_matrix(y_test, preds)
    sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap='Blues',
                xticklabels=['Loss','Win'], yticklabels=['Loss','Win'])
    ax.set_title(f"{LABELS[t]}\n({best_name})")
    ax.set_xlabel('Predicted'); ax.set_ylabel('Actual')
plt.tight_layout(); plt.show()


## Feature Importance (Random Forest)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
for ax, t in zip(axes.flat, TRADERS):
    rf = results[t]['Random Forest']['model']
    imp = pd.Series(rf.feature_importances_, index=FEATURES).sort_values(ascending=True)
    imp.plot(kind='barh', ax=ax, color='#2196F3', alpha=0.85)
    ax.set_title(f'{LABELS[t]} — RF Feature Importances')
    ax.set_xlabel('Importance'); ax.grid(axis='x', alpha=0.3)
plt.tight_layout(); plt.show()


## Summary

> These baselines establish a performance floor. All models will be compared against these benchmarks in subsequent notebooks.

In [ ]:
summary = []
for t in TRADERS:
    for name in MODELS:
        r = results[t][name]
        summary.append({'Trader': LABELS[t], 'Model': name,
                        'Val Acc': round(r['acc'], 4), 'Val AUC': round(r['auc'], 4)})

df_summary = pd.DataFrame(summary)
print(df_summary.to_string(index=False))
